In [ ]:

import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
#from azure.search.documents.models import VectorizedQuery

service_endpoint = 'https://csrabogenai.search.windows.net' #os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = 'vrm-index2' #os.environ["AZURE_SEARCH_INDEX_NAME"]
key = 'x9q4J48TFwp4ago80LhWjjibGjzyTR1ztLVsJN8BKqAzSeAy9AGI' #os.environ["AZURE_SEARCH_API_KEY"]

OpenAIEndpoint = 'https://openai-rabogenai.openai.azure.com/'
OpenAIKey = 'e91bf1f85ffd4f8b8ec5c7d87fcc1bc6'

def get_document_info():

    import json
    import uuid
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.text_splitter import CharacterTextSplitter

    loader = PyPDFLoader(r"D:\Python\input\Constitution.pdf")
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
    documents = text_splitter.split_documents(documents)

    docs = []
    for doc in documents:
        
        docs.append({"documentID":str(uuid.uuid4()),"content":doc.page_content,"embedding":get_embeddings(doc.page_content)})
        
    json_data=json.dumps(docs)
    
    with open(r"D:\Python\output\HandbookContent.json","w") as f:
        f.write(json_data)

    with open(r"D:\Python\output\HandbookContent.json","r") as f:
        document = json.load(f)

    return document


def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    import openai

    open_ai_endpoint = OpenAIEndpoint #os.getenv("OpenAIEndpoint")
    open_ai_key = OpenAIKey #os.getenv("OpenAIKey")

    client = openai.AzureOpenAI(
        azure_endpoint=open_ai_endpoint,
        api_key=open_ai_key,
        api_version="2023-09-01-preview",
    )
    embedding = client.embeddings.create(input=[text], model="embeddingmodel-rabogenai")
    return embedding.data[0].embedding


def get_hotel_index(name: str):
    from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchProfile,
        HnswAlgorithmConfiguration,
    )

    fields = [
        SimpleField(name="documentID", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_profile_name="my-vector-config",
        )
    ]

    vector_search = VectorSearch(
        profiles=[VectorSearchProfile(name="my-vector-config", algorithm_configuration_name="my-algorithms-config")],
        algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
    )
    return SearchIndex(name=name, fields=fields, vector_search=vector_search)


if __name__ == "__main__":
    print("Started")
    credential = AzureKeyCredential(key)
    index_client = SearchIndexClient(service_endpoint, credential)
    index = get_hotel_index(index_name)
    index_client.create_index(index)
    print("Index Config")
    client = SearchClient(service_endpoint, index_name, credential)
    hotel_docs = get_document_info()
    print("Doc Loaded")
    #print(hotel_docs)
    client.upload_documents(documents=hotel_docs)
    print("Completed")

In [172]:
import openai

CogAIEndpoint = 'https://acsrabogenai.search.windows.net'
CogAIKey = 'Twy7mcmNDtpSyXgwIWPFNFiMahgK1Xh5ibl6Z8mVJgAzSeBjlsNU'

search_client = SearchClient(CogAIEndpoint, index_name, AzureKeyCredential(CogAIKey))

query = "What is the Qualifications for election as President"
vector = VectorizedQuery(vector=get_embeddings(query),k_nearest_neighbors=2,fields="embedding")

results = search_client.search(
    vector_queries=[vector],
    select=["content"]
)

input_text=""
for result in results:
    input_text = input_text + result['content'] + " "



In [ ]:
#openai.api_type = "azure"
#openai.api_base = "https://aoairabogenai.openai.azure.com/"
#openai.api_version = "2024-02-01"
#openai.api_key = "b1d16fbce07644668935410f4c54f74e"

open_ai_endpoint = OpenAIEndpoint #os.getenv("OpenAIEndpoint")
open_ai_key = OpenAIKey #os.getenv("OpenAIKey")

client = openai.AzureOpenAI(
    azure_endpoint=open_ai_endpoint,
    api_key=open_ai_key,
    api_version="2023-09-01-preview",
)

completion = client.completions.create(
    model = 'openai-rabogenai',
    prompt = f"Answer Input:{input_text}. Question:{query}",
    #max_tokens=10,
    #top_p=1,
    #frequency_penalty=0,
    #presence_penalty=0
)      

completion.choices[0].text



In [ ]:
print(completion.choices[0].text)